# Test whether impropers are properly implemented in SMIRFF

(Material from this will be migrated to a standard test once I work through it).
By D. Mobley, 9/19/16.

Initial testing on benzene and o-xylene. 

In [138]:
# Import stuff we need
from smarty import *
import openeye.oechem as oechem
import openeye.oeiupac as oeiupac
import openeye.oeomega as oeomega

In [139]:
# Load our forcefield 
#ffxml = '../../../utilities/convert_frosst/smirff99Frosst.ffxml'
ffxml = 'smirff99Frosst_modified.ffxml'
ff = ForceField(ffxml)

In [140]:
# Initialize o-xylene and benzene as a test molecules
molnames = ['benzene', 'o-xylene']
oemols = []
topologies = []
systems = []
for molname in molnames:
    mol = oechem.OEMol()
    oeiupac.OEParseIUPACName(mol, molname)
    omega = oeomega.OEOmega()
    omega(mol)
    # Generate coordinates as we'll need them later for energy evaluations
    omega = oeomega.OEOmega()
    omega(mol)
    oechem.OETriposAtomNames(mol) #Assign Tripos names
    oechem.OETriposAtomTypes(mol) #Assign tripos numeric types
    oechem.OETriposTypeNames(mol) #Assign tripos text types
    oemols.append(oechem.OEMol(mol))
    # Generate topology and system
    topology = generateTopologyFromOEMol(mol)
    topologies.append(topology)
    print("Creating system for %s..." % molname)
    system = ff.createSystem(topology, [mol], chargeMethod = 'OECharges_AM1BCCSym', verbose = True)
    systems.append(system)

Creating system for benzene...
Applying oechem.oequacpac.OEAssignPartialCharges with specified charge method "OECharges_AM1BCCSym".

HarmonicBondGenerator:

                                                     [*:1]~[*:2] :       24 matches
                                               [#6X4:1]-[#6X4:2] :        0 matches
                                               [#6X4:1]-[#6X3:2] :        0 matches
                                      [#6X4:1]-[#6X3:2]=[#8X1+0] :        0 matches
                                               [#6X3:1]-[#6X3:2] :        0 matches
                                               [#6X3:1]:[#6X3:2] :       12 matches
                                               [#6X3:1]=[#6X3:2] :        0 matches
                                                   [#6:1]-[#7:2] :        0 matches
                                               [#6X3:1]-[#7X3:2] :        0 matches
                               [#6X4:1]-[#7X3:2]-[#6X3]=[#8X1+0] :        0 matches
   

## Now generate mol2 files for our molecules with parm99 types so we can parameterize with AMBER
We'll just go through and modify benzene and toluene to get parm99 types and then write them out. 

In [141]:
# Next step -- need AMBER prmtop/crd files for o-xylene and/or benzene for comparison
# I should be able to get these by hand-typing them with parm@frosst types and then generating from the frcmod, etc.

# Benzene uses CA for carbons, HA for hydrogens. 
# o-xylene would be the same for the ring atoms, and then HC for the methyl hydrogen and CT for the carbon. 
# Superficially that looks consistent with SMIRKS patterns in SMIRFF99Frosst

oemols_tripos = [] # Store with original types
# Create benzene and o-xylene and swap atom names before writing right here! 
for idx, oemol in enumerate(oemols):
    oemols_tripos.append(oechem.OEMol(oemol))
    for atom in oemol.GetAtoms():
        typename= atom.GetType()
        #print(typename)
        if typename=='H':
            # What is it connected to?
            c_ar = False
            for bond in atom.GetBonds():
                nbr = bond.GetNbr(atom)
                #print('Atom type %s has neighbor type %s' % (typename, nbr.GetType()))
                if nbr.GetType=='C.ar' or nbr.GetType()=='CA':
                    c_ar = True
            if c_ar:
                atom.SetType('HA')
            else: 
                atom.SetType('HC')
        elif typename=='C.ar':
            atom.SetType('CA')
        elif typename=='C.3':
            atom.SetType('CT')
        #print("   New type %s..." % atom.GetType())

    # Write resulting molecule
    ostream = oemolostream(molnames[idx]+'.mol2')
    ostream.SetFlavor(oechem.OEFormat_MOL2, oechem.OEOFlavor_MOL2_Forcefield) # Use forcefield flavor to preserve types
    OEWriteMolecule( ostream, oemol)
    ostream.close()


### Generate AMBER prmtop and crd files for these

In [142]:
frcmod = 'parm_Frosst.frcmod'
from tleap_tools import *
for (idx, molname) in enumerate(molnames):
    prmtop = molname+'.prmtop'
    mol2file = molname+'.mol2'
    crd = molname+'.crd'
    run_tleap(molname, mol2file, frcmod, prmtop, crd, verbose = False)

## Compare energies of molecules and their components

In [143]:
from smarty.forcefield_utils import *
import numpy as np
import simtk.unit as unit

for idx, molname in enumerate(molnames):
    print("Examining %s..." % molname)
    oemol = oemols_tripos[idx]
    prmtop = molname+'.prmtop'
    crd = molname+'.crd'
    frosst_component, smirff_component, frosst_energy, smirff_energy = compare_molecule_energies(prmtop, crd, ff, oemol, skip_assert = True, verbose = False)
    for comp in frosst_component:
        cdiff = np.abs( frosst_component[comp] - smirff_component[comp])
        #print cdiff, comp
        if cdiff > 1e-5*unit.kilocalorie_per_mole:
            print("   Component %s differs by %s..." % (comp, cdiff.in_units_of(unit.kilocalorie_per_mole)))
    
print("Processed.")
    

Examining benzene...
   Component torsion differs by 2.27541048199e-05 kcal/mol...
Examining o-xylene...
   Component torsion differs by 6.37419156656e-05 kcal/mol...
Processed.


## Compare actual parameters

In [144]:
# We can use the same code as the above but just switch the assertion about checking terms to on
for idx, molname in enumerate(molnames):
    print("Examining %s..." % molname)
    oemol = oemols_tripos[idx]
    prmtop = molname+'.prmtop'
    crd = molname+'.crd'
    frosst_component, smirff_component, frosst_energy, smirff_energy = compare_molecule_energies(prmtop, crd, ff, oemol, skip_assert = False, verbose = True)

    
print("Processed.")

Examining benzene...
Doing an AM1 calculation to get Wiberg bond orders.

HarmonicBondGenerator:

                                                     [*:1]~[*:2] :       24 matches
                                               [#6X4:1]-[#6X4:2] :        0 matches
                                               [#6X4:1]-[#6X3:2] :        0 matches
                                      [#6X4:1]-[#6X3:2]=[#8X1+0] :        0 matches
                                               [#6X3:1]-[#6X3:2] :        0 matches
                                               [#6X3:1]:[#6X3:2] :       12 matches
                                               [#6X3:1]=[#6X3:2] :        0 matches
                                                   [#6:1]-[#7:2] :        0 matches
                                               [#6X3:1]-[#7X3:2] :        0 matches
                               [#6X4:1]-[#7X3:2]-[#6X3]=[#8X1+0] :        0 matches
                                    [#6X3:1](=[#8X1+0])-[#7X3:

AssertionError: Error:  (improper) PeriodicTorsionForce entry (2, 0, 4, 8) has different numbers of terms (1 and 6, respectively).

In [145]:
# Currently having issues with improper involving [*:1]~[#6X3:2](~[*:3])~[*:4] which has "different number of terms"
# i.e. for atoms 2, 0, 4, 8 which are CA-CA-CA-H

# Covered by <Improper smirks="[*:1]~[#6X3:2](~[*:3])~[*:4]" id="i1" k1="1.1" periodicity1="2" phase1="180."/>
# I think this is the equivalent of X -X -CA-HA which is 1.1          180.          2. in the parm99 file. 
# Seems to be an exact translation of that.

# So, what does the "different number of terms" mean? Applied a different number of times? That would be OK. 
# Ah, it's the number of terms involving that improper in openmoltools.system_checker. 
# We EXPECT six terms where there was previously one so that's OK, but we will need to reduce barriers by factor of six.
# Current question -- do we do this internally in SMARTY or in the forcefield file?
# See discussion at https://github.com/open-forcefield-group/smarty/issues/51